<a href="https://colab.research.google.com/github/k1lowM/sd-on-colabo/blob/main/controlnet_automatic1111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# 念のためコンテンツルートに移動
%cd /content

# torchのバージョンをwebui用に変更
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

# webuiをインストール
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

# sd-webui-controlnet のダウンロード
!git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet

# sd_dreambooth_extension のダウンロード
dreambooth = False
if dreambooth:
  !git clone https://github.com/d8ahazard/sd_dreambooth_extension.git /content/stable-diffusion-webui/extensions/sd_dreambooth_extension

# 作業ディレクトリを移動させておく
%cd /content/stable-diffusion-webui

# モデルダウンロード関数の定義
!apt-get -y install -qq aria2 # aria2を使うと爆速で、モデルをダウンロードできる
def download_model(download_model_url, output_dir):
  # urlの最後がモデル名として、ファイル名を取得する
  file_name_path = download_model_url.split('/')[-1]
  # ダウンロードの実行
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_model_url} -d {output_dir} -o {file_name_path}
  # ダウンロード後の配置されたディレクトリを一応確認
  !ls -lh {output_dir}

modelsDir = "/content/stable-diffusion-webui/models/Stable-diffusion"

# stable-diffsion v1.4 のモデルのダウンロード
download_model("https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt", modelsDir)

# v1.4以外の学習済みモデルをダウンロード
learnedModel = False    # True = controlnetが動く，False = controlnetが動かない

if learnedModel:
  # stable-diffsion v2.1 model(768×768)のダウンロード
  #download_model("https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors", modelsDir)
  #!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

  # stable-diffsion v2.1 base model(512×512)のダウンロード
  download_model("https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors", modelsDir)
  !wget "https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml" -O {modelsDir}/v2-1_512-ema-pruned.yaml"

  # dreamlike-art
  download_model("https://huggingface.co/dreamlike-art/dreamlike-photoreal-2.0/resolve/main/dreamlike-photoreal-2.0.safetensors", modelsDir)

  # basil_mix
  download_model("https://huggingface.co/nuigurumi/basil_mix/resolve/main/Basil_mix_fixed.safetensors", modelsDir)
  download_model("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors", "/content/stable-diffusion-webui/models/VAE")

  # Realism Engine 1.0
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/20414?type=Model&format=SafeTensor" -d {modelsDir} -o "realismEngine_v10.safetensors"
  !wget "https://civitai.com/api/download/models/20414?type=Config&format=Other" -O {modelsDir}/realismEngine_v10.yaml

  # nfixer & nrealfixer for Illuminati Diffusion v1.1
  !wget "https://civitai.com/api/download/models/15921" -O /content/stable-diffusion-webui/embeddings/nfixer.pt
  !wget "https://civitai.com/api/download/models/15927" -O /content/stable-diffusion-webui/embeddings/nrealfixer.pt
  
  # ChilloutMix
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://civitai.com/api/download/models/11745" -d {modelsDir} -o "chilloutmix_NiPrunedFp32Fix.safetensors"

# sd-webui-controlnetモデルをダウンロード
controlnet = True    # True = controlnetが動く，False = controlnetが動かない
controlnetHeavy = False
controlnetLight = True

controlnetModelsDir = "/content/stable-diffusion-webui/models/ControlNet"
if controlnet:
  # 重量版
  if controlnetHeavy:
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_canny.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_depth.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_hed.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_mlsd.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_normal.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_openpose.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_scribble.pth", controlnetModelsDir)
    download_model("https://huggingface.co/lllyasviel/ControlNet/resolve/main/models/control_sd15_seg.pth", controlnetModelsDir)

  # 軽量版をダウンロード
  if controlnetLight:
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors", controlnetModelsDir)

    # T2I-Adapter
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_canny-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_color-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_depth-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_keypose-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_openpose-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_seg-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_sketch-fp16.safetensors", controlnetModelsDir)
    download_model("https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/t2iadapter_style-fp16.safetensors", controlnetModelsDir)

# webuiを別URLで立ち上げる(ngrok User:k1low ,Pass:1111)
!python launch.py --ngrok 2Nxt6QScUNwn4wpWmJF4r688TDn_6wA1FXVVtMb18ceH1U96P --ngrok-region jp --gradio-debug --gradio-auth k1low:1111 --xformers --enable-insecure-extension-access --ckpt {modelsDir}/sd-v1-4-full-ema.ckpt

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu117
fatal: destination path 'stable-diffusion-webui' already exists and is not an empty directory.
fatal: destination path '/content/stable-diffusion-webui/extensions/sd-webui-controlnet' already exists and is not an empty directory.
/content/stable-diffusion-webui

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
a942dc|OK  |       0B/s|/content/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4-full-ema.ckpt

Status Legend:
(OK):download completed.
total 7.2G
-rw-r--r-- 1 root root    0 Apr  4 14:17 'Put Stable Diffusion checkpoints here.txt'
-rw-r--r-- 1 root root 7.2G Apr  4 14:18  sd-v1-4-full-ema.ckpt

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
c37c59|OK  |   385MiB/s

In [ ]:
# GoogleドライブをGoogle Colab にマウント
# (Googleドライブ使わないなら実行の必要なし)

# from google.colab import drive
# drive.mount('/content/drive')